In [2]:
!pip install pyvi

import tensorflow as tf
import pandas as pd 
import numpy as np
from string import digits
from collections import Counter
from pyvi import ViTokenizer
from gensim.models.word2vec import Word2Vec
from keras.utils.np_utils import to_categorical
%matplotlib inline

     |████████████████████████████████| 8.5 MB 22.5 MB/s 
     |████████████████████████████████| 743 kB 55.0 MB/s 


In [7]:
from google.colab import drive
drive.mount('/content/drive')

data_train = pd.read_csv("/content/drive/MyDrive/Learning/Natural Language Processing/Exercises/cnn/vlsp_sentiment_train.csv", sep='\t')
data_train.columns =['Class', 'Data']
print("Trainning sample\n")
data_train.head()
data_test = pd.read_csv("/content/drive/MyDrive/Learning/Natural Language Processing/Exercises/cnn/vlsp_sentiment_test.csv", sep='\t')
data_test.columns =['Class', 'Data']
print("Test sample\n")
data_test.tail()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Trainning sample

Test sample



,Class,Data
1045,0,30 củ à :)
1046,0,Apple bán dc thi samsung cũng lời nhiêu. Ng...
1047,0,có thể giúp android vượt trội so với ios chớ c...
1048,0,Mẹ mình từng sang Đài Loan và có mua 1 cái iph...
1049,0,Tùng Minh Nguyễn điện thoại của vk bị như này ...


In [8]:
print(data_train.shape)
print(data_test.shape)

(5100, 2)
(1050, 2)


In [36]:
labels = data_train.iloc[:, 0].values
reviews = data_train.iloc[:, 1].values
print(reviews)

['Mình đã dùng anywhere thế hệ đầu, quả là đầy thất vọng, hiện tại đang vứt xó. Giá thì đắt, ngốn pin như ăn gỏi, nặng'
 'Quan tâm nhất là độ trễ có cao không, dùng thi thoảng nó cứ trễ bực mình, đấy mới chỉ là dùng văn phòng chứ game thì chắc là ném đi từ lâu. Không biết con này có độ trễ không nhỉ. Dùng nhiều loại nhưng vẫn kết nhất con chuột sứ mitsumi, gọn bấm nảy tốt'
 'dag xài con cùi bắp 98k....pin trâu, mỗi tội đánh liên minh ức chế đập hết 2 con'
 ...
 'Dùng oppo mà bộ nhớ 4gb thì k chơi games đc đâu.hệ thống đã chiếm 2,2gb rồi.chuyển sag thẻ sd thì cứ như k sag vậy :('
 'Sao tui thích xài hàng oppo mà lựa toàn mấy đứa tui ghét quảng cáo vậy trời'
 'mới mở hộp ,oy mở vào camera mà đã có ảnh chụp trc đó ở góc màn hình oy.... thấy sai sai']


In [10]:
encoded_labels = []

for label in labels:
    if label == -1:
        encoded_labels.append([1,0,0])
    elif label == 0:
        encoded_labels.append([0,1,0])
    else:
        encoded_labels.append([0,0,1])

encoded_labels = np.array(encoded_labels)  

In [11]:
reviews_processed = []
unlabeled_processed = [] 
for review in reviews:
    review_cool_one = ''.join([char for char in review if char not in digits])
    reviews_processed.append(review_cool_one)

In [12]:
#Use PyVi for Vietnamese word tokenizer
word_reviews = []
all_words = []
for review in reviews_processed:
    review = ViTokenizer.tokenize(review.lower())
    word_reviews.append(review.split())
   

In [13]:
EMBEDDING_DIM = 400 # how big is each word vector
MAX_VOCAB_SIZE = 10000 # how many unique words to use (i.e num rows in embedding vector)
MAX_SEQUENCE_LENGTH = 300 # max number of words in a comment to use

In [15]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical


In [16]:
tokenizer = Tokenizer(num_words=MAX_VOCAB_SIZE, lower=True, char_level=False)
tokenizer.fit_on_texts(word_reviews)
sequences_train = tokenizer.texts_to_sequences(word_reviews)
word_index = tokenizer.word_index


In [17]:
data = pad_sequences(sequences_train, maxlen=MAX_SEQUENCE_LENGTH)
labels = encoded_labels

In [18]:
print('Shape of X train and X validation tensor:',data.shape)
print('Shape of label train and validation tensor:', labels.shape)

Shape of X train and X validation tensor: (5100, 300)
Shape of label train and validation tensor: (5100, 3)


In [22]:
import gensim
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess

from gensim.models.keyedvectors import KeyedVectors

word_vectors = KeyedVectors.load_word2vec_format('/content/drive/MyDrive/Learning/Natural Language Processing/Exercises/cnn/vi-model-CBOW.bin', binary=True)


vocabulary_size=min(len(word_index)+1,MAX_VOCAB_SIZE)
embedding_matrix = np.zeros((vocabulary_size, EMBEDDING_DIM))
for word, i in word_index.items():
    if i>=MAX_VOCAB_SIZE:
        continue
    try:
        embedding_vector = word_vectors[word]
        embedding_matrix[i] = embedding_vector
    except KeyError:
        embedding_matrix[i]=np.random.normal(0,np.sqrt(0.25),EMBEDDING_DIM)

del(word_vectors)

from keras.layers import Embedding
embedding_layer = Embedding(vocabulary_size,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            trainable=True)

In [24]:
from keras.layers import Dense, Input, GlobalMaxPooling1D
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.models import Model
from keras.layers import Input, Dense, Embedding, Conv2D, MaxPooling2D, Dropout,concatenate
from keras.layers.core import Reshape, Flatten
from keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from keras.models import Model
from keras import regularizers
sequence_length = data.shape[1]
filter_sizes = [3,4,5]
num_filters = 100
drop = 0.5

inputs = Input(shape=(sequence_length,))
embedding = embedding_layer(inputs)
reshape = Reshape((sequence_length,EMBEDDING_DIM,1))(embedding)

conv_0 = Conv2D(num_filters, (filter_sizes[0], EMBEDDING_DIM),activation='relu',kernel_regularizer=regularizers.l2(0.01))(reshape)
conv_1 = Conv2D(num_filters, (filter_sizes[1], EMBEDDING_DIM),activation='relu',kernel_regularizer=regularizers.l2(0.01))(reshape)
conv_2 = Conv2D(num_filters, (filter_sizes[2], EMBEDDING_DIM),activation='relu',kernel_regularizer=regularizers.l2(0.01))(reshape)

maxpool_0 = MaxPooling2D((sequence_length - filter_sizes[0] + 1, 1), strides=(1,1))(conv_0)
maxpool_1 = MaxPooling2D((sequence_length - filter_sizes[1] + 1, 1), strides=(1,1))(conv_1)
maxpool_2 = MaxPooling2D((sequence_length - filter_sizes[2] + 1, 1), strides=(1,1))(conv_2)

merged_tensor = concatenate([maxpool_0, maxpool_1, maxpool_2], axis=1)
flatten = Flatten()(merged_tensor)
reshape = Reshape((3*num_filters,))(flatten)
dropout = Dropout(drop)(flatten)
output = Dense(units=3, activation='softmax',kernel_regularizer=regularizers.l2(0.01))(dropout)


# this creates a model that includes
model = Model(inputs, output)


adam = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])
model.summary()

#define callbacks
early_stopping = EarlyStopping(monitor='val_loss', min_delta=0.01, patience=4, verbose=1)
callbacks_list = [early_stopping]


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 300)]        0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 300, 400)     3167600     input_1[0][0]                    
__________________________________________________________________________________________________
reshape (Reshape)               (None, 300, 400, 1)  0           embedding[0][0]                  
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 298, 1, 100)  120100      reshape[0][0]                    
______________________________________________________________________________________________

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [26]:
model.fit(data, labels, validation_split=0.2,
          epochs =5, batch_size=256,callbacks=callbacks_list, shuffle=True, verbose=2)

Epoch 1/5
16/16 - 103s - loss: 0.8174 - accuracy: 0.5412 - val_loss: 1.3891 - val_accuracy: 0.0000e+00
Epoch 2/5
16/16 - 101s - loss: 0.6355 - accuracy: 0.6745 - val_loss: 1.1447 - val_accuracy: 0.0422
Epoch 3/5
16/16 - 101s - loss: 0.5617 - accuracy: 0.7353 - val_loss: 1.0603 - val_accuracy: 0.1098
Epoch 4/5
16/16 - 101s - loss: 0.5144 - accuracy: 0.7801 - val_loss: 1.1838 - val_accuracy: 0.0559
Epoch 5/5
16/16 - 101s - loss: 0.4851 - accuracy: 0.8083 - val_loss: 1.1668 - val_accuracy: 0.0784


In [27]:
labels_test = data_test.iloc[:, 0].values
reviews_test = data_test.iloc[:, 1].values

In [28]:
encoded_labels_test = []

for label_test in labels_test:
    if label_test == -1:
        encoded_labels_test.append([1,0,0])
    elif label_test == 0:
        encoded_labels_test.append([0,1,0])
    else:
        encoded_labels_test.append([0,0,1])

encoded_labels_test = np.array(encoded_labels_test)  

In [29]:
reviews_processed_test = []
unlabeled_processed_test = [] 
for review_test in reviews_test:
    review_cool_one = ''.join([char for char in review_test if char not in digits])
    reviews_processed_test.append(review_cool_one)

In [30]:
#Use PyVi for Vietnamese word tokenizer
word_reviews_test = []
all_words = []
for review_test in reviews_processed_test:
    review_test = ViTokenizer.tokenize(review_test.lower())
    word_reviews_test.append(review_test.split())

In [31]:
sequences_test = tokenizer.texts_to_sequences(word_reviews_test)
data_test = pad_sequences(sequences_test, maxlen=MAX_SEQUENCE_LENGTH)
labels_test = encoded_labels_test

In [32]:
print('Shape of X train and X validation tensor:',data_test.shape)
print('Shape of label train and validation tensor:', labels_test.shape)

Shape of X train and X validation tensor: (1050, 300)
Shape of label train and validation tensor: (1050, 3)


In [33]:
score = model.evaluate(data_test, labels_test)

33/33 [==============================] - 5s 166ms/step - loss: 0.6635 - accuracy: 0.6124


In [34]:
print("%s: %.2f%%" % (model.metrics_names[0], score[0]*100))
print("%s: %.2f%%" % (model.metrics_names[1], score[1]*100))


loss: 66.35%
accuracy: 61.24%


Testing Model

In [41]:
test = "Hãy yêu nhau đi"
reviews_processed_test = []

review_not_contain_digit = ''.join([char for char in test if char not in digits])
reviews_processed_test.append(review_not_contain_digit)

word_reviews_test = []

for review in reviews_processed_test:
    review = ViTokenizer.tokenize(review.lower())
    word_reviews_test.append(review.split())


tokenizer = Tokenizer(num_words=MAX_VOCAB_SIZE, lower=True, char_level=False)
tokenizer.fit_on_texts(word_reviews_test)
sequences_train = tokenizer.texts_to_sequences(word_reviews_test)

sampleToPredit = pad_sequences(sequences_train, maxlen=MAX_SEQUENCE_LENGTH)

class_names = ["Negative", "Neutral", "Positive"]
pre = model.predict(sampleToPredit)
print(class_names[np.argmax(pre)])

Positive


In [44]:
test = "Dấu tình sầu"
reviews_processed_test = []

review_not_contain_digit = ''.join([char for char in test if char not in digits])
reviews_processed_test.append(review_not_contain_digit)

word_reviews_test = []

for review in reviews_processed_test:
    review = ViTokenizer.tokenize(review.lower())
    word_reviews_test.append(review.split())


tokenizer = Tokenizer(num_words=MAX_VOCAB_SIZE, lower=True, char_level=False)
tokenizer.fit_on_texts(word_reviews_test)
sequences_train = tokenizer.texts_to_sequences(word_reviews_test)

sampleToPredit = pad_sequences(sequences_train, maxlen=MAX_SEQUENCE_LENGTH)

class_names = ["Negative", "Neutral", "Positive"]
pre = model.predict(sampleToPredit)
print(class_names[np.argmax(pre)])

Positive
